# SMS Corpus Preprocessing

Download and clean dataset. Optional as the cleaned dataset is already provided in corpus folder.

In [1]:
!wget -O ../data/smsCorpus_en.json.zip https://storage.googleapis.com/nlp-singlish-manglish/smsCorpus_en_2015.03.09_all.json.zip

--2020-08-28 08:13:10--  https://storage.googleapis.com/nlp-singlish-manglish/smsCorpus_en_2015.03.09_all.json.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 74.125.200.128
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2535167 (2.4M) [application/zip]
Saving to: ‘../data/smsCorpus_en.json.zip’

../data/smsCorpus_e 100%[===================>]   2.42M  9.94MB/s    in 0.2s    

2020-08-28 08:13:10 (9.94 MB/s) - ‘../data/smsCorpus_en.json.zip’ saved [2535167/2535167]



In [2]:
!unzip -o -d ../data ../data/smsCorpus_en.json.zip

Archive:  ../data/smsCorpus_en.json.zip
  inflating: ../data/smsCorpus_en_2015.03.09_all.json  


In [3]:
import json
import pandas as pd

In [4]:
with open("../data/smsCorpus_en_2015.03.09_all.json") as f:
    sms = json.loads(f.read())

In [5]:
sms_df = pd.DataFrame(sms["smsCorpus"]["message"])

In [6]:
sms_df.shape

(55835, 6)

In [7]:
sms_df["user_id"] = sms_df.source.apply(lambda s: s["srcNumber"]["$"])

In [8]:
sms_df.user_id.value_counts()

23249055a638bbc9b1fc5eb7dac9b4259524183451bc74bc                    4951
0                                                                   4754
975bd06869dccbc3b74b999a840f68a537795b6c8b833dfe                    3976
c3e3c793bbdb71d520fa190b5c23b4cf796b52d96b40d9d938cc3db68eb0e909    3383
df501e0b21bc88cc1e464dd95bf722da8865851c93267498                    2621
                                                                    ... 
40                                                                     1
39                                                                     1
38                                                                     1
132                                                                    1
48                                                                     1
Name: user_id, Length: 343, dtype: int64

In [9]:
sms_df["text"] = sms_df.text.apply(lambda s: s["$"])

In [10]:
# drop duplicated text
sms_df.drop_duplicates(subset=["text"], inplace=True)

In [11]:
# filter users with at least 10 smses
sms_df = sms_df.groupby("user_id").filter(lambda x: len(x) >= 10)

In [12]:
sms_df.shape

(48092, 7)

In [13]:
# remap user id from 0 to N
user_id_remap = {value:ix for ix, value in enumerate(sms_df.user_id.unique())}

In [14]:
sms_df.user_id.replace(user_id_remap, inplace=True)

In [15]:
sms_df.user_id.nunique()

199

In [16]:
sms_df.shape

(48092, 7)

In [17]:
sms_df[["user_id", "text"]].to_csv("../corpus/sg_sms_corpus_en.csv", index=False)